In [7]:
import scipy.misc
import glob
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder



# load own handwriting
dataset = []

# load the png image data
for image_file_name in glob.glob('/home/pratyush/Documents/nepali_characters/numerals/*/*.jpg'):
    label = int(image_file_name[-12:-11])
    #data=pd.read_csv("/home/pratyush/Documents/labels_num.csv")
    
    
    # load image data from png files into an array
    #print ("loading ... ", image_file_name)
    img_array = scipy.misc.imread(image_file_name, flatten=True)  #Read an image from a file as an array
    
    # reshape from 28x28 to list of 1024 values, invert values
    img_data  = 255.0 - img_array.reshape(1296)
    
    # then scale data to range from 0.01 to 1.0
    img_data = (img_data / 255.0*0.99 +0.01)
    #print(numpy.min(img_data))
    #print(numpy.max(img_data))    
    # append label and image data  to test data set
    record = np.append(img_data,label)
    dataset.append(record)

    
#start
data= np.array(dataset)
np.random.shuffle(data)
df=pd.DataFrame(data)
df.to_csv("data.csv")
data_set=pd.read_csv("data.csv")
#print(data_set)

#note: .iloc is necessary while using panda dataframe file

#one hot encoding
onehotencoder = OneHotEncoder()

#initailization
X=np.array(data_set.iloc[:,1:1297])
Y=np.array(data_set.iloc[:,1297])
b=1
#Y_reshape=Y.reshape(-1,1)
#Y_one= onehotencoder.fit_transform(Y_reshape)
#print(Y_one)
print(Y.shape)

#splitting
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
W=np.zeros((1296))
W_T=W.T
print("shape of X train is",X_train.shape)
print("shape of Y train is",y_train.shape)

learning_rate=0.03
epochs=200
lda=0.1


def get_hypo(X,W,b):
    temp= (np.matmul(X,W)+b)
    h=1/(1+np.exp(-temp))
    return h

def cost_func(X,Y,W,b):
    Y_=1-Y
    
    cost=-(np.multiply(Y,np.log(get_hypo(X,W,b)))+np.multiply(Y_,np.log(1-get_hypo(X,W,b))))
    total_cost=(1/len(Y)) * np.sum(cost)
    print("total loss",total_cost)

    

    
def gradient_step(X,Y,W,b,learning_rate):
    h=get_hypo(X,W,b)
    cost=cost_func(X,Y,W,b)
    temp=h-Y
    gradient_weights=(1/ len(Y)) * ((np.matmul(temp,X))+(lda/len(Y))*np.sum(W**2))
    gradient_bias=(1/len(Y)) * np.sum(temp)
    new_w = W - learning_rate*gradient_weights
    new_b = b - learning_rate*gradient_bias
    return [new_w,new_b]
       
       
def gradient_descent_runner(X,Y,W,b,learning_rate,epochs):
    final_weight=[]
    final_bias=[]
    
    
    for j in range(10):
        Y_train_one = ((Y==j)).astype(int)
        final_w=W
        final_b=b
        for i in range(epochs):
            [final_w,final_b] = gradient_step(X,Y_train_one,final_w,final_b,learning_rate)
            if(i==(epochs-1)):
                final_weight.append(final_w)
                final_bias.append(final_b)
                print("optimal values of weight for value ",j," is ",final_w)
                print("optimal values of bias for value ",j, "is ",final_b)
                
    return final_weight,final_bias
    #     plt.scatter(Y,plot)
    #     plt.xlabel("hypothesis value")
    #     plt.ylabel("actual value")

    
def test(X,final_weight,final_bias):
    temp=[]
    for i in range (10):
        value=get_hypo(X,final_weight[i,:],final_bias[i])
        print(value)
        temp.append(value)
    return temp

def accuracy_test(X,Y,final_weight,final_bias):
    count=0
    accuracy=0
    for i in range (len(Y)):
        hypo=test(X[i,:],final_weight,final_bias)
        hypo=np.array(hypo)
        max_hypo=np.where(hypo==np.amax(hypo))
        if(max_hypo==Y[i]):
            count=count+1
    accuracy=(count/len(Y))*100
    print("accuracy for the model is ",accuracy)
    
    



value,bias=gradient_descent_runner(X_train,y_train,W,b,learning_rate,epochs)    
final_weight=np.array(value)
final_bias=np.array(bias)



hypo=test(X_test[20,:],final_weight,final_bias)
hypo=np.array(hypo)
#print("final value of hypothesis ",hypo)
max_hypo=np.where(hypo==np.amax(hypo))
print("max hypothesis is ",max_hypo)

image =X_test[20,:]
scipy.misc.imsave("/home/pratyush/predict2.png",image.reshape(-1,36))

accuracy_test(X_test,y_test,final_weight,final_bias)







/home/pratyush/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


(2880,)
shape of X train is (2304, 1296)
shape of Y train is (2304,)
total loss 1.2099630764071116
total loss 0.6164853896678977
total loss 0.48112083495147806
total loss 0.43070682562962187
total loss 0.4012120882598722
total loss 0.3794551135900363
total loss 0.36141911538026333
total loss 0.34558586600732066
total loss 0.3312957403752088
total loss 0.31822636181501857
total loss 0.3061982729742503
total loss 0.295095115153691
total loss 0.2848292172983519
total loss 0.27532670333370707
total loss 0.2665213127654217
total loss 0.2583520788398236
total loss 0.2507626059956315
total loss 0.24370090834384506
total loss 0.23711936289224242
total loss 0.23097461452193632
total loss 0.22522739677496403
total loss 0.21984228196104152
total loss 0.2147873874041842
total loss 0.21003406309506834
total loss 0.20555657979544387
total loss 0.20133183013325798
total loss 0.19733904995246973
total loss 0.19355956339938896
total loss 0.18997655272830485
total loss 0.18657485227166504
total loss 0.1

total loss 0.06078632431550164
total loss 0.060687727665286254
total loss 0.06058963828094521
total loss 0.06049205167388645
total loss 0.06039496341152984
total loss 0.06029836911640803
total loss 0.060202264465284816
total loss 0.060106645188290696
total loss 0.060011507068075017
total loss 0.0599168459389748
total loss 0.05982265768619918
total loss 0.05972893824502983
total loss 0.05963568360003648
total loss 0.05954288978430745
total loss 0.05945055287869474
total loss 0.05935866901107355
optimal values of weight for value  0  is  [-0.01966726 -0.02937855 -0.03069337 ... -0.03117057 -0.03102984
 -0.01311998]
optimal values of bias for value  0 is  0.7729568937284277
total loss 1.2116991875182228
total loss 0.568368900502089
total loss 0.4270838443060434
total loss 0.3763261201002706
total loss 0.34840641835916314
total loss 0.3290412516278663
total loss 0.3136896192014967
total loss 0.30055195425607184
total loss 0.28881250171187367
total loss 0.27806572053946993
total loss 0.2680

total loss 0.03841570826575281
total loss 0.03832602212569018
total loss 0.038236884026753515
total loss 0.03814828855378808
total loss 0.038060230364871765
total loss 0.03797270419006118
total loss 0.03788570483016374
total loss 0.037799227155535076
total loss 0.03771326610490107
total loss 0.03762781668420415
total loss 0.03754287396547302
total loss 0.03745843308571538
total loss 0.03737448924583314
total loss 0.037291037709559396
total loss 0.037208073802416974
total loss 0.037125592910697634
total loss 0.037043590480461845
total loss 0.036962062016558356
total loss 0.03688100308166324
total loss 0.03680040929533792
total loss 0.0367202763331058
total loss 0.0366405999255469
total loss 0.03656137585741033
total loss 0.03648259996674401
total loss 0.03640426814404122
total loss 0.03632637633140382
total loss 0.03624892052172147
total loss 0.03617189675786676
total loss 0.036095301131905594
total loss 0.03601912978432289
total loss 0.035943378903262796
total loss 0.03586804472378342


total loss 0.14258611254350237
total loss 0.14244626620662018
total loss 0.14230712839654588
total loss 0.14216869217102074
total loss 0.14203095068814572
total loss 0.14189389720447568
total loss 0.1417575250731576
total loss 0.14162182774211265
total loss 0.14148679875225992
total loss 0.14135243173578155
total loss 0.1412187204144275
total loss 0.14108565859785896
total loss 0.1409532401820299
total loss 0.14082145914760513
total loss 0.14069030955841397
total loss 0.14055978555993895
total loss 0.14042988137783824
total loss 0.1403005913165011
total loss 0.14017190975763533
total loss 0.1400438311588861
total loss 0.13991635005248523
total loss 0.13978946104392984
total loss 0.13966315881069027
total loss 0.13953743810094593
total loss 0.1394122937323484
total loss 0.13928772059081168
total loss 0.1391637136293278
total loss 0.13904026786680837
total loss 0.1389173783869508
total loss 0.13879504033712806
total loss 0.13867324892730248
total loss 0.1385519994289624
total loss 0.1384

...

/home/pratyush/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:156: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


0.019853313978588402
0.0100383446631291
0.006844991973402318
0.016519904787003548
0.17512329336786572
0.2176704746916454
0.004219121204604708
0.006265701273650447
0.000886254256813968
0.3241597161365312
0.3967953120760648
0.008067191750863015
0.31487798718421095
0.11847957959838709
0.009325513711853597
0.007397546410600368
0.004875943192735223
0.12817880947438562
0.23719373636101776
0.6516041790584596
0.45826235760207007
0.08963255823295023
0.5808350089332862
0.057329998922258346
0.129383574234463
0.03705696453745732
0.17648457574246054
0.11366413718982268
0.00011342442166931648
0.004194794374691712
0.023976076979977955
0.0007595793734602804
9.772462629842951e-05
0.21757587598863287
0.7287518726748681
0.0005046422999378158
0.0037466441373227534
0.9705463006804866
0.0035362509729984326
0.00033875227416433804
0.0018135557966469486
0.00017654688030320788
9.147414492052477e-05
0.008139770154761833
0.1407940593843983
0.004809847424594486
0.0003938878623302528
0.750340401140783
0.00172951945

0.08269897981701665
0.05647606245239357
0.07829551423085107
0.20322562047738574
0.01677936524852418
0.008932172383351379
0.25489833530612105
0.1864900424312205
0.04979592609603351
0.561812413535524
0.013128199295608357
0.008000770219604181
0.002423997196811221
0.020934793616934372
0.00024106545523813248
0.00048364767896866804
0.008358466509209603
0.01912765964656943
0.3969955230233056
0.32817871100159407
0.0039042847974390987
0.03140280831477665
0.003244633136584139
0.017366375605769844
0.004385939371755098
0.0051771467343655
0.025379453738797502
0.06964307441481829
0.001296324664963547
0.0008107843386513584
0.02321148434819176
0.0004941983939476688
0.9746003912131483
0.03421338341891355
0.001197770566871855
0.007757968849409245
0.02156943561182158
0.013140506732257907
0.7753879458271101
0.3688004331475863
0.020375202986907214
0.0237601028248534
0.011381059904831901
0.07538844946129437
0.006075652045672388
0.0009329552855334938
0.010282557036952378
0.4548626835089599
0.0007035217584399

0.03331626671615999
0.02895968429012568
0.04981497577183889
0.28173229588354537
0.0014905830781221647
0.0037650649770987732
0.006235194979015785
0.0289374254243216
0.006218035451475988
0.0018174667379344612
0.7009285969215543
0.0036580047723561116
0.0021572147499799824
0.00023483858628191472
0.0001435765026571392
0.002391783508406536
0.027437544717181368
0.4497794648341972
0.253728701653651
0.0028228541001125867
0.01307929300694014
0.01173526619897374
0.02225908184243815
0.004954871112679356
0.9686147713074434
0.006733283733161947
0.008020209294598351
0.038143987082812324
0.01154957922459435
0.003343487503687968
0.000864491574649965
0.004701572287158062
0.007829333498294403
0.0004957207807327802
0.02810301091338179
0.003662694114804571
0.009651924479184747
0.8283631409032834
0.11029763308943093
0.010335041410255754
0.12095621222722705
0.019905597374594918
0.0926795808380533
0.0005435714168308793
0.0008876039395437924
0.04963066190776772
0.050638304325704146
0.32939481696974954
0.387286

0.017821578862774277
0.010753045942184436
2.8217958094825442e-05
0.00032082829480076523
0.07639734831138616
0.0018380297068841494
0.0009693817588319891
0.016882657743516
0.07619817913854142
0.01609132486827173
0.02564120156066934
0.026056598379597493
0.6397763164109398
0.1559572011807968
0.17931577204920446
0.0715624684921766
0.09787826050394677
0.045872501262619575
0.0003225382024846464
0.9509866111078802
0.006998762557796331
0.030004317442708532
0.01604948391368046
0.004303381289094503
0.006240354752290836
0.000983790554961318
0.0038978985589356503
0.08393000265052222
0.9818191446283938
0.00028936825930209995
0.00030584470990975
0.00028327654528614464
5.0111736260559236e-05
5.174107009677708e-05
0.001174642629107
0.011638288162618471
0.024369868238475995
0.00042925445433313183
0.8299850494635566
0.004227226233397297
8.768615387933808e-05
0.0006744607507864568
0.0003524608172214056
3.944872537951759e-05
0.0031022187704411023
0.027711718614159404
0.0089133287434061
2.109902976721349e-0

In [111]:
image =X_test[20,:]

In [112]:
scipy.misc.imsave("/home/pratyush/predict2.png",image.reshape(-1,36))

/home/pratyush/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  """Entry point for launching an IPython kernel.
